## Data Fetching Examples

### 1- Import the DataFetcher

In [16]:
from torchtrade.components import Market,Broker,TradeDirection
import numpy as np
from datetime import datetime

In [12]:
# Setting params
symbols = ["BTC/USDT","ETH/USDT","ADA/USDT"]
timeframe = "15m"

In [13]:
market = Market(symbols,timeframe, since = datetime(2023,1,1),include_technical_indicators=True)
market.prepare()
broker = Broker(market,0.98,0.001)

/ Fetching 15m OHLCV Data from binance for BTC/USDT, ETH/USDT, ADA/USDT
\ Data Fetched Successfully!
Source    TimeFrame    Symbols                       Since                Until                  Timestamps Count
--------  -----------  ----------------------------  -------------------  -------------------  ------------------
binance   15m          BTC/USDT, ETH/USDT, ADA/USDT  2023-01-01 12:15:00  2023-03-02 00:00:00                5712
Checking market Data
Market Data Check successful


In [26]:
#Dummy strategy that buys randomly 
market.reset()
broker.reset()
while not market.market_ended():
    r = np.random.random(1),
    i = np.random.randint(0,len(symbols))
    if r[0] < 0.01:
        broker.openTrade("long",symbols[i], TradeDirection.LONG, 1,1, stop_loss=0.02 , risk_reward=1.5)
    market.next()
        

In [28]:
for trade in broker.trades:
    print(trade)


Last Update Timestamp: 2023-01-02 07:00:00
Symbol: ADA/USDT
Trade Details:
    Execution time: 2023-01-01 15:30:00
    Closing Time: 2023-01-02 07:00:00
    Entry Price: 0.2461
    Time In Trade : 62
    Stop Loss %: 0.02
    Risk Reward: 1.5
    Stop Loss Price: 0.241178
    Take Profit Price: 0.253483
    Status: TradeStatus.CLOSED
    Realized P&L %: 0.027970000000000002
    Realized P&L : 0.006883417000000001
    Realized Discounted P&L% : 0.007992987969615103
    Unrealized P&L %: 0
    Unrealized P&L : 0
    Unrealized Discounted P&L% : 0
    Paid Commission : 0.0004995830000000001
            

Last Update Timestamp: 2023-01-04 03:00:00
Symbol: ADA/USDT
Trade Details:
    Execution time: 2023-01-02 07:15:00
    Closing Time: 2023-01-04 03:00:00
    Entry Price: 0.2529
    Time In Trade : 175
    Stop Loss %: 0.02
    Risk Reward: 1.5
    Stop Loss Price: 0.247842
    Take Profit Price: 0.260487
    Status: TradeStatus.CLOSED
    Realized P&L %: 0.027970000000000036
    Realized